In [1]:
import csv
import requests

PROMETHEUS_URL = 'http://10.6.0.5:12101'
PROMETHEUS_SOURCE_NAME = 'node_exporter:9100'
HOURS  = 72

QUERY = f"100-rate(node_cpu_seconds_total{{mode='idle',instance='{PROMETHEUS_SOURCE_NAME}'}}[10s])*100"
QUERY_TEMP = f'node_hwmon_temp_celsius{{instance="{PROMETHEUS_SOURCE_NAME}",job="node"}}'
STEP = 10
SINGLE_RANGE = 60 * 60 * 1
TIMES = HOURS

In [2]:
all_data = {}
temp_data = {}

def info_for_key(key: str):
    if key not in all_data or len(all_data[key]) == 0:
        return (None, None, 0)
    first = max(el[0] for el in all_data[key])
    last = min(el[0] for el in all_data[key])
    count = len(all_data[key])
    return first, last, count

In [3]:
import time

now = int(time.time())
for i in reversed(range(TIMES)):
    start = now - SINGLE_RANGE*(i+1)
    end = now - SINGLE_RANGE*i
    res = requests.get(f'{PROMETHEUS_URL}/api/v1/query_range',  params={'query': QUERY,
                    'start': start, 'end': end, 'step': STEP})
    res2 = requests.get(f'{PROMETHEUS_URL}/api/v1/query_range',  params={'query': QUERY_TEMP,
                    'start': start, 'end': end, 'step': STEP})
    results = res.json()['data']['result']
    results_temp = res2.json()['data']['result']
    for result in results:
        key = str(result['metric']['cpu'])
        if not key in all_data:
            all_data[key] = [] 
        all_data[key].extend(result['values'])
    for result in results_temp:
        key = str(result['metric']['sensor'])
        if not key in temp_data:
            temp_data[key] = [] 
        temp_data[key].extend(result['values'])



In [4]:
print(len(all_data['0']))
info_for_key('0')

23500


(1652607988, 1652373418, 23500)

In [16]:
with open('cpu_measures.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['cpu', 'timestamp', 'usage'])
    for key in all_data:
        for el in all_data[key]:
            writer.writerow([key, *el])
with open('temp_measures.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['sensor', 'timestamp', 'temperature'])
    for key in temp_data:
        for el in temp_data[key]:
            writer.writerow([key, *el])